In [5]:
# Download Chinook SQLite DB from GitHub
!wget https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite -O chinook.db

# Confirm it's downloaded
import os
assert os.path.exists('chinook.db')
print("✅ chinook.db is ready!")

--2025-06-01 20:25:48--  https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite [following]
--2025-06-01 20:25:48--  https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1067008 (1.0M) [application/octet-stream]
Saving to: ‘chinook.db’

chinook.db          100%[===================>]   1.02M  --.-KB/s    in 0.0

In [6]:
### useful: functions for displaying results from sql queries using pandas
from IPython.display import display
import pandas as pd

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)


In [7]:
# Step 1: Import necessary modules
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker

In [8]:
# Step 2: Create engine to connect to the chinook.db file
engine = sqlalchemy.create_engine('sqlite:///chinook.db')

In [9]:
# Step 3: Connect to the database and assign the connection to cur
cur = engine.connect()

In [10]:
### useful: extract classes from the chinook database
metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

## we need to do this once
from sqlalchemy.ext.automap import automap_base

# produce a set of mappings from this MetaData.
Base = automap_base(metadata=metadata)

# calling prepare() just sets up mapped classes and relationships.
Base.prepare()

# also prepare an orm session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [11]:
# Print all table names in the database
print(metadata.tables.keys())

dict_keys(['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'Track', 'MediaType', 'Playlist', 'PlaylistTrack'])


In [12]:
# Access the Track class
Track = Base.classes.Track

# Query the first 3 tracks
tracks = session.query(Track).limit(3).all()

# Print track names
for t in tracks:
    print(t.Name)

For Those About To Rock (We Salute You)
Balls to the Wall
Fast As a Shark


In [13]:
# Access related tables
Album = Base.classes.Album
Track = Base.classes.Track

# Join Track and Album tables
results = session.query(Track.Name, Album.Title).join(Album).limit(20).all()

# Print track names and their album titles
for name, title in results:
    print(f"{name} — {title}")

For Those About To Rock (We Salute You) — For Those About To Rock We Salute You
Balls to the Wall — Balls to the Wall
Fast As a Shark — Restless and Wild
Restless and Wild — Restless and Wild
Princess of the Dawn — Restless and Wild
Put The Finger On You — For Those About To Rock We Salute You
Let's Get It Up — For Those About To Rock We Salute You
Inject The Venom — For Those About To Rock We Salute You
Snowballed — For Those About To Rock We Salute You
Evil Walks — For Those About To Rock We Salute You
C.O.D. — For Those About To Rock We Salute You
Breaking The Rules — For Those About To Rock We Salute You
Night Of The Long Knives — For Those About To Rock We Salute You
Spellbound — For Those About To Rock We Salute You
Go Down — Let There Be Rock
Dog Eat Dog — Let There Be Rock
Let There Be Rock — Let There Be Rock
Bad Boy Boogie — Let There Be Rock
Problem Child — Let There Be Rock
Overdose — Let There Be Rock


In [14]:
# Access InvoiceItem and Track
InvoiceItem = Base.classes.InvoiceLine  # sometimes called InvoiceLine in Chinook
Track = Base.classes.Track

# Join InvoiceLine with Track to get names and quantity
results = session.query(Track.Name, InvoiceItem.Quantity)\
    .join(Track, Track.TrackId == InvoiceItem.TrackId)\
    .limit(10).all()

# Display results
for name, qty in results:
    print(f"{name} — Quantity Sold: {qty}")

Balls to the Wall — Quantity Sold: 1
Restless and Wild — Quantity Sold: 1
Put The Finger On You — Quantity Sold: 1
Inject The Venom — Quantity Sold: 1
Evil Walks — Quantity Sold: 1
Breaking The Rules — Quantity Sold: 1
Dog Eat Dog — Quantity Sold: 1
Overdose — Quantity Sold: 1
Love In An Elevator — Quantity Sold: 1
Janie's Got A Gun — Quantity Sold: 1


In [15]:
# Group by TrackId, count total quantity sold, join with Track to get name
from sqlalchemy import func

results = session.query(Track.Name, func.sum(InvoiceItem.Quantity).label('total_sold'))\
    .join(InvoiceItem, Track.TrackId == InvoiceItem.TrackId)\
    .group_by(Track.TrackId)\
    .order_by(func.sum(InvoiceItem.Quantity).desc())\
    .limit(10).all()

# Print top 10
for name, total in results:
    print(f"{name} — Sold: {total}")

Balls to the Wall — Sold: 2
Inject The Venom — Sold: 2
Snowballed — Sold: 2
Overdose — Sold: 2
Deuces Are Wild — Sold: 2
Not The Doctor — Sold: 2
Por Causa De Você — Sold: 2
Welcome Home (Sanitarium) — Sold: 2
Snowblind — Sold: 2
Cornucopia — Sold: 2


In [16]:
# Access needed tables
Artist = Base.classes.Artist
Album = Base.classes.Album
Track = Base.classes.Track
InvoiceItem = Base.classes.InvoiceLine

# Join all needed tables to aggregate total sales per artist
results = session.query(Artist.Name, func.sum(InvoiceItem.Quantity).label("total_sold"))\
    .join(Album, Artist.ArtistId == Album.ArtistId)\
    .join(Track, Album.AlbumId == Track.AlbumId)\
    .join(InvoiceItem, Track.TrackId == InvoiceItem.TrackId)\
    .group_by(Artist.ArtistId)\
    .order_by(func.sum(InvoiceItem.Quantity).desc())\
    .limit(10).all()

# Print top 10 artists by sales
for name, total in results:
    print(f"{name} — Sold: {total}")

Iron Maiden — Sold: 140
U2 — Sold: 107
Metallica — Sold: 91
Led Zeppelin — Sold: 87
Os Paralamas Do Sucesso — Sold: 45
Deep Purple — Sold: 44
Faith No More — Sold: 42
Lost — Sold: 41
Eric Clapton — Sold: 40
R.E.M. — Sold: 39
